In [1]:
import pandas
import numpy as np
import os.path as osp
from general_tools.in_out.basics import files_in_subdirs
from general_tools.in_out.basics import pickle_data, unpickle_data

In [173]:
from language_3d_io import GeoWordDatum, GeoWordData, Language_Game_Data_Loader, GeoWordsDataSet, load_glove_pretrained_model

In [172]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
top_in_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/'
dialogues_dir = osp.join(top_in_dir, 'pilot_chair_data/chatMessage')

In [5]:
close_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/closest_nn/data/model_names_of_triplets.txt'
split_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/one_far/data/model_names_of_triplets.txt'
far_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/both_far/data/model_names_of_triplets.txt'

In [55]:
triplets_to_sn_models = Language_Game_Data_Loader.triplet_id_to_shape_net_model_ids(close_map_file, far_map_file, split_map_file)
dialogue_files = [f for f in files_in_subdirs(dialogues_dir, '.csv')]
language_data = []
for f in dialogue_files:
    language_data.append(Language_Game_Data_Loader.load_dialogues_of_game(f))

In [75]:
geo_embedding = pandas.read_pickle('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_chair_embeddings.pkl')
word_emb_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/glove.6B/glove.6B.100d.txt'
pretrained_word_emb = load_glove_pretrained_model(word_emb_file)

Loading glove model.
Done. 400000  words loaded!


In [155]:
all_text = '' 
for i, game in enumerate(language_data):
    for _, dialogues in game.iteritems():
        all_text += ' ' + dialogues['dialogue']
        
replace_dic = {'!': ' ', ',': ' ', '?': ' ', '.': ' ', ';': ' ', '(': ' ', ')': ' ', '/': ' ',
               '&': 'and', '"': ' ', '-': ' ', '=': ' ', '*': ' ','\\': '', '\'': ' ',
               
               '\'t': ' not', '\'s': ' of',
              
              'SpEaKeR:' :' ', 'lIsTeNeR:' :' '
              }

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

In [156]:
all_text = replace_all(all_text, replace_dic)
all_text = all_text.lower()
words = all_text.split()

In [176]:
from collections import Counter
from general_tools.simpletons import invert_dictionary

counts = Counter(words)
all_vocab = sorted(counts, key=counts.get, reverse=True)
vocab = [v for v in all_vocab if counts[v] > 1]

word_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
word_to_int['#START#'] = 0
counts['#START#'] = 1000

int_to_word = invert_dictionary(word_to_int)

def bias_init_word_vec(word_counts, int_to_word):
    bias_init_vector = np.array([1.0 * word_counts[int_to_word[i]] for i in int_to_word])
    bias_init_vector /= np.sum(bias_init_vector) 
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector)
    return bias_init_vector

bv = bias_init_word_vec(counts, int_to_word)

pickle_data('word_context.pickle', word_to_int, int_to_word, bv)

In [177]:
# NEW ONLY FOR INT (Re-write.)
words_missed = set()
words_used = set()
triplet_to_data = dict()

mlen = 0
for i, game in enumerate(language_data):
    for triplet_id, dialogues in game.iteritems():
        triplet_condition, triplet_num = triplet_id.split('_')
        a, b, c = triplets_to_sn_models[triplet_condition][triplet_num]
        triplet_geo_emb = geo_embedding.loc[[a, b, c]].as_matrix().T
        
        index = np.arange(3)        
        triplet_geo_emb = triplet_geo_emb[:, index]
        label = np.where(index==0)[0][0]        
        
        words_as_ints = []
            
        for w in replace_all(dialogues['dialogue'], replace_dic).split():
            wl = w.lower()
            if wl in word_to_int:                
                words_as_ints.append(word_to_int[wl])
                words_used.add(wl)
            else: # TODO deal with uknown words
                words_missed.add(wl)
        
        if len(words_as_ints) >= 1:                
            if triplet_id not in triplet_to_data:
                triplet_to_data[triplet_id] = []
            triplet_to_data[triplet_id].append(GeoWordDatum(words_as_ints, triplet_geo_emb, label, triplet_id))

In [30]:
# with open ('all_ling_data.txt', 'w') as fout:
#     for game in language_data:
#         for key in game.keys():
#             fout.write(game[key]['dialogue'] + '\n')
# Use tf.nn.embedding_lookup instead and map words to integers


# Deal with "speaker:", "listener:", misspelled words
# dentist's chair
# from string import punctuation

# map numbers to language
# typos

# sqare -> square 
# sqiggle -> squiggly
# shorest -> shortest
# pilow -> pillow

words_missed = set()
words_embedded = set()
triplet_to_data = dict()

for i, game in enumerate(language_data):
    for triplet_id, dialogues in game.iteritems():
        triplet_condition, triplet_num = triplet_id.split('_')
        a, b, c = triplets_to_sn_models[triplet_condition][triplet_num]
        triplet_geo_emb = geo_embedding.loc[[a, b, c]].as_matrix().T
        
        index = np.arange(3)
        np.random.shuffle(index)
        
        triplet_geo_emb = triplet_geo_emb[:, index]
        label = np.where(index==0)[0][0]
        
        triplet_word_seq = []        
        for w in replace_all(dialogues['dialogue'], replace_dic).split():
            wl = w.lower()
            if wl in word_with_emb:
                triplet_word_seq.append(word_with_emb[wl])
                words_embedded.add(wl)
            else: # TODO deal with uknown words
                words_missed.add(wl)
        
        if len(triplet_word_seq) >= 1:
            if triplet_id not in triplet_to_data:
                triplet_to_data[triplet_id] = []
            triplet_to_data[triplet_id].append(GeoWordDatum(triplet_word_seq, triplet_geo_emb, label, triplet_id))

In [178]:
def train_test_split(triplet_to_data, single_condition=None):
    n_train = 130
    n_test = 10
    relevant_triplets = []
    for key in triplet_to_data.keys():
        if single_condition is not None:
            if key.split('_')[0] == single_condition:
                relevant_triplets.append(key)
        else:
            relevant_triplets.append(key)
    
    relevant_triplets = np.array(relevant_triplets, dtype=object)
    np.random.shuffle(relevant_triplets)
    
    tr_triplets = relevant_triplets[:n_train]
    test_triplets = relevant_triplets[n_train:]    
    train_data = {}
    test_data = {}
    
    for key in tr_triplets:
        train_data[key] = triplet_to_data[key]
        
    for key in test_triplets:
        test_data[key] = [triplet_to_data[key][0]] # JUST ONE 
    
    return train_data, test_data

In [179]:
train_dic, test_dic = train_test_split(triplet_to_data)

In [180]:
word_matrix, geo_matrix, label_matrix, mask_matrix, cond_matrix = GeoWordData.organize_into_numpy_arrays(train_dic, max_seq_len=15)
gw_data = GeoWordsDataSet(words=word_matrix, geometries=geo_matrix, labels=label_matrix, masks=mask_matrix, conditions=cond_matrix)
pickle_data('train_130_all_cond_max_seq_15_.pkl', gw_data)

word_matrix, geo_matrix, label_matrix, mask_matrix, cond_matrix = GeoWordData.organize_into_numpy_arrays(test_dic, max_seq_len=15)
gw_data = GeoWordsDataSet(words=word_matrix, geometries=geo_matrix, labels=label_matrix, masks=mask_matrix, conditions=cond_matrix)
pickle_data('test_10_all_cond_max_seq_15_.pkl', gw_data)

array(['close_45', 'close_45', 'close_44', 'close_44', 'close_42',
       'close_42', 'close_41', 'close_41', 'close_40', 'close_40',
       'far_138', 'far_138', 'far_138', 'far_138', 'far_138', 'far_138',
       'far_138', 'far_39', 'far_35', 'far_35', 'far_35', 'far_35',
       'far_31', 'far_30', 'far_30', 'far_30', 'far_30', 'far_30',
       'far_128', 'far_128', 'far_128', 'far_128', 'far_122', 'far_122',
       'far_122', 'far_122', 'far_123', 'far_123', 'far_123', 'close_6',
       'close_6', 'close_7', 'close_7', 'close_7', 'close_7', 'close_7',
       'close_7', 'close_7', 'close_7', 'close_4', 'close_4', 'close_4',
       'close_4', 'close_4', 'close_4', 'close_4', 'close_4', 'split_31',
       'split_31', 'split_31', 'split_31', 'split_31', 'split_31',
       'close_2', 'close_2', 'close_2', 'close_2', 'close_2', 'close_2',
       'close_2', 'close_2', 'close_2', 'close_2', 'close_2', 'close_0',
       'close_0', 'close_8', 'close_8', 'close_8', 'close_8', 'close_8',
      

In [163]:
test_dic

{'close_1': [<language_3d_io.GeoWordDatum instance at 0x7f3022c95cf8>],
 'close_10': [<language_3d_io.GeoWordDatum instance at 0x7f30198950e0>],
 'close_100': [<language_3d_io.GeoWordDatum instance at 0x7f3022bf1710>],
 'close_101': [<language_3d_io.GeoWordDatum instance at 0x7f30199d4f80>],
 'close_104': [<language_3d_io.GeoWordDatum instance at 0x7f30199ef098>],
 'close_105': [<language_3d_io.GeoWordDatum instance at 0x7f3022bf1680>],
 'close_109': [<language_3d_io.GeoWordDatum instance at 0x7f3022bbb3b0>],
 'close_110': [<language_3d_io.GeoWordDatum instance at 0x7f30199eecf8>],
 'close_112': [<language_3d_io.GeoWordDatum instance at 0x7f3019a3e4d0>],
 'close_114': [<language_3d_io.GeoWordDatum instance at 0x7f30198abb48>],
 'close_116': [<language_3d_io.GeoWordDatum instance at 0x7f3019a0c758>],
 'close_117': [<language_3d_io.GeoWordDatum instance at 0x7f30198ab710>],
 'close_119': [<language_3d_io.GeoWordDatum instance at 0x7f3019a0cf38>],
 'close_12': [<language_3d_io.GeoWordDatu

In [33]:
word_matrix, geo_matrix, label_matrix = GeoWordData.organize_into_numpy_arrays(train_dic, max_seq_len=10)
gw_data = GeoWordsDataSet(words=word_matrix, geometries=geo_matrix, labels=label_matrix)
pickle_data('train_100_close_max_seq_10_.pkl', gw_data)

In [35]:
word_matrix, geo_matrix, label_matrix = GeoWordData.organize_into_numpy_arrays(test_dic, max_seq_len=10)
gw_data = GeoWordsDataSet(words=word_matrix, geometries=geo_matrix, labels=label_matrix)
pickle_data('test_40_close_max_seq_10.pkl', gw_data)